In [155]:
import numpy as np
from qiskit_optimization import QuadraticProgram
from qiskit.quantum_info import SparsePauliOp
from qiskit_optimization.converters import InequalityToEquality, IntegerToBinary, LinearEqualityToPenalty
from docplex.mp.model import Model

# SciPy minimizer routine
from scipy.optimize import minimize

In [156]:
class QuadraticProgram2Ising:
    """A mapping of `QuadraticProgram` to a Ising Hamiltonian
    in `SparsePauliOp` format
    """
    def __init__(self):
        self.input_type = QuadraticProgram
        self.output_type = SparsePauliOp

    def run(self, program):
        if not isinstance(program, self.input_type):
            raise TypeError(f"Program is invalid input type, must be {self.input_type}")
        ising, constant = program.to_ising(opflow=False)
        return ising + SparsePauliOp.from_list([("I"*ising.num_qubits, constant)])

In [157]:
import copy

class CompositeWorkflow:
    def __init__(self, passes):
        self.passes = passes
    
    def run(self, input):
        temp = copy.deepcopy(input)
        for individual_pass in self.passes:
            temp = individual_pass.run(temp)
        return temp

In [158]:
mdl = Model("docplex model")
x = mdl.binary_var("x")
y = mdl.binary_var("y")
z = mdl.binary_var("z")
mdl.minimize(x - 2*y + 3*z + x*y + x*z + 2*y*z)
#print(mdl.export_as_lp_string())

In [181]:
[mdl.get_var_by_index(kk) for kk in range(mdl.number_of_variables)]

[docplex.mp.Var(type=B,name='x'),
 docplex.mp.Var(type=B,name='y'),
 docplex.mp.Var(type=B,name='z')]

In [159]:
qp = QuadraticProgram.from_docplex_mp(mdl)

In [216]:
cw = CompositeWorkflow([InequalityToEquality(), # Transformation
                        IntegerToBinary(), # Transformation
                        LinearEqualityToPenalty(), # Transformation
                        #QuadraticProgram2Ising() # Conversion
                        ])

In [217]:
qubo = cw.run(qp)

In [211]:
ising

SparsePauliOp(['IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI', 'III'],
              coeffs=[-1.  +0.j,  0.25+0.j, -2.25+0.j,  0.25+0.j,  0.25+0.j,  0.5 +0.j,
  2.  +0.j])

In [163]:
from qiskit.circuit.library import QAOAAnsatz

In [164]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Sampler, Session

In [165]:
service = QiskitRuntimeService()

KeyboardInterrupt: 

In [ ]:
backend = service.get_backend('ibmq_qasm_simulator')

In [ ]:
ansatz = QAOAAnsatz(ising, reps=2)

In [ ]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return cost


In [ ]:
session = Session(backend=backend)

estimator = Estimator(session=session, options={"shots": int(1e4)})
sampler = Sampler(session=session, options={"shots": int(1e4)})


In [ ]:
x0 = 2 * np.pi * np.random.rand(ansatz.num_parameters)

In [ ]:
res = minimize(cost_func, x0, args=(ansatz, ising, estimator), method="COBYLA")

In [ ]:
res

In [150]:
# Assign solution parameters to ansatz
qc = ansatz.assign_parameters(res.x)
# Add measurements to our circuit
qc.measure_all()


In [151]:
# Sample ansatz at optimal parameters
samp_dist = sampler.run(qc, shots=int(1e4)).result().quasi_dists[0]
# Close the session since we are now done with it
session.close()

In [152]:
samp_dist

{5: 0.0028,
 1: 0.1073,
 2: 0.4162,
 6: 0.0111,
 4: 0.0142,
 0: 0.3038,
 3: 0.1094,
 7: 0.0352}

In [195]:
qubo

<QuadraticProgram: minimize x*y + x*z + 2*y*z + x - 2*y + 3*z, 3 variables, 0 constraints, 'docplex model'>

In [191]:
from qiskit_optimization.problems import LinearExpression, QuadraticObjective

In [206]:
dok = QuadraticObjective(qubo).linear.

In [207]:
qubo.objective.evaluate(samp_dist)

IndexError: column index (5) out of range

In [212]:
from qiskit.utils import algorithm_globals
from qiskit.algorithms.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)
from qiskit_optimization import QuadraticProgram
from qiskit.visualization import plot_histogram
from typing import List, Tuple
import numpy as np

/var/folders/01/dp6k619n4dng8l43hj8z2nl80000gn/T/ipykernel_62507/3887331713.py:2: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver


In [213]:
algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA(), initial_point=[0.0, 0.0])
exact_mes = NumPyMinimumEigensolver()

In [214]:
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

In [221]:
exact_result = qaoa.solve(qubo)
print(exact_result)

fval=-2.0, x=0.0, y=1.0, z=0.0, status=SUCCESS
